In [21]:
textgrid_dir = "/Users/joregan/Playing/textgrids_shared"
timecode_dir = "/Users/joregan/timecode_cut"
output_dir = "/tmp/textgrid_cut"

In [22]:
from pathlib import Path

textgrid_path = Path(textgrid_dir)
timecode_path = Path(timecode_dir)
output_path = Path(output_dir)

In [23]:
def get_timecode_offsets(filename):
    if type(filename) == Path:
        filename = str(filename)

    with open(filename) as inf:
        lines = [l.strip() for l in inf.readlines()]
        assert lines[0] == ",Frame,Time (Seconds),TimeCode", f"CSV file ({filename}) seems to be incorrect"
        p_start = lines[1].split(",")
        start = float(p_start[2])
        p_end = lines[-1].split(",")
        end = float(p_end[2])
        return start, end

In [27]:
from praatio import textgrid
from praatio.utilities.constants import Interval

def modify_textgrid(filename, start, end, output=None):
    if type(filename) == Path:
        filename = str(filename)
    if not output:
        output = filename
    elif type(output) == Path:
        output = str(output)

    tg = textgrid.openTextgrid(filename, True)
    new_tg = textgrid.Textgrid()
    for tiername in tg.tierNames:
        tier = tg.getTier(tiername)
        new_entries = []
        for entry in tier.entries:
            if entry[0] < start and entry[1] < start:
                continue
            elif entry[0] > end and entry[1] > end:
                continue
            elif entry[0] <= start and entry[1] > start:
                if entry[2] != "":
                    print("Warning: truncating entry", filename, entry)
                if new_entries == []:
                    new_entries.append(Interval(0.0, entry[1] - start, entry[2]))
                else:
                    print("Shouldn't have existing entries!!", entry)
                    new_entries.append(Interval(entry[0] - start, entry[1] - start, entry[2]))
            elif entry[0] >= start and entry[1] <= end:
                new_entries.append(Interval(entry[0] - start, entry[1] - start, entry[2]))
            elif entry[0] <= end and entry[1] > end:
                if entry[2] != "":
                    print("Warning: truncating entry", filename, entry)
                new_entries.append(Interval(entry[0] - start, entry[1] - start, entry[2]))
            else:
                print("There should be no default case", entry)
        tier_start = new_entries[0][0]
        tier_end = new_entries[-1][1]
        new_tier = textgrid.IntervalTier(tiername, new_entries, tier_start, tier_end)
        new_tg.addTier(new_tier)
    tg.save(output, format="long_textgrid", includeBlankSpaces=True)

In [28]:
if not output_path.is_dir():
    output_path.mkdir()

for tgfile in textgrid_path.glob("*.[Tt]ext[Gg]rid"):
    base_stem = tgfile.stem
    stem = base_stem
    if "_inter" in stem:
        stem = stem.replace("_inter", "")
    else:
        stem = stem.replace("_main", "")

    csv_file = timecode_path / f"{stem}.csv"

    if not csv_file.exists():
        print("No timecode for", tgfile)
        continue

    start, end = get_timecode_offsets(csv_file)
    output_file = output_path / f"{base_stem}.TextGrid"
    modify_textgrid(tgfile, start, end, output_file)


Maximum timestamp in Textgrid changed from (762.8836) to (762.9205)
Maximum timestamp in Textgrid changed from (342.37899999999996) to (342.4133)
Maximum timestamp in Textgrid changed from (326.3109) to (326.6203)
Maximum timestamp in Textgrid changed from (400.8546) to (402.711)
Maximum timestamp in Textgrid changed from (510.08019999999993) to (510.109)
No timecode for /Users/joregan/Playing/textgrids_shared/hsi_7_0719_209_002_main.TextGrid
Maximum timestamp in Textgrid changed from (289.7103) to (289.9525)
Maximum timestamp in Textgrid changed from (510.16799999999995) to (514.441)
